In [1]:
import os
import pandas as pd
import geopandas as gpd
import fiona
import random
from rasterio.plot import  show, show_hist
from rasterio.mask import mask
from shapely.geometry import Polygon
from matplotlib import pyplot
import rasterio
import glob
import os
import time
import numpy as np

In [5]:
in_dir = '/Users/jonathanburton/Desktop/Fall2020/Geog5092/lab5/data'
out_dir = '/Users/jonathanburton/Desktop/Fall2020/Geog5092/lab5/output'
lab5functions  = (r'/Users/jonathanburton/Desktop/Fall2020/Geog5092/lab5/lab5functions.py')
from lab5functions import *

dem_rs = rasterio.open(in_dir + '/bigElk_dem.tif')
dem_arr = dem_rs.read(1)
sat_imgs = glob.glob(r'/Users/jonathanburton/Desktop/Fall2020/Geog5092/lab5/data/L5_big_elk/*.tif')
sat_imgs.sort()


DEM = slopeAspect(dem_arr, 90)
DEM_aspect = reclassAspect(dem_arr)
DEM_reclass = reclassByHisto(dem_arr, 10) 

with rasterio.open(in_dir + '/fire_perimeter.tif') as fire:
    fire_per = fire.read()
     
band3 = []
band4 = []
final_print = []
year = []
flat_rr = []

for img in sat_imgs:
    if 'B3.tif' in img:
        band3.append(img)
    if 'B4.tif' in img:
        band4.append(img)
    
for b3, b4 in zip(band3, band4):
    year.append(b3[-11:-7])
    with rasterio.open(b3, 'r') as data3:
        band3_arr = data3.read()
    with rasterio.open(b4, 'r') as data4:
        band4_arr = data4.read()
    ndvi = (band4_arr - band3_arr) / (band4_arr + band3_arr)
    healthy_veg = np.where(fire_per == 2)
    burned_veg = np.where(fire_per == 1)
    ndvi_mean = ndvi[healthy_veg].mean()
    rr = ndvi / ndvi_mean
    final = rr[burned_veg].mean()
    final_print.append(final)
    flat = rr.flatten()
    flat_rr.append(flat)
stacked_ratio = np.vstack(flat_rr)
trend_line = np.polyfit(range(10), stacked_ratio, 1)[0]
trend_reshape = trend_line.reshape(280, 459)
mean_coef = np.where(fire_per==1, trend_reshape, np.nan)
coefficient = np.nanmean(mean_coef)
for yr, rat in zip(year, final_print):
    print("In", yr, "the mean recovery ratio was", rat)
print("The mean coefficient of recovery across all years for the burned area is", coefficient)

In 2002 the mean recovery ratio was 0.41126525
In 2003 the mean recovery ratio was 0.5412664
In 2004 the mean recovery ratio was 0.51346225
In 2005 the mean recovery ratio was 0.61524826
In 2006 the mean recovery ratio was 0.7161738
In 2007 the mean recovery ratio was 0.70540994
In 2008 the mean recovery ratio was 0.739514
In 2009 the mean recovery ratio was 0.7126317
In 2010 the mean recovery ratio was 0.58509773
In 2011 the mean recovery ratio was 0.6258852
The mean coefficient of recovery across all years for the burned area is 0.02179563271729749


In [6]:
def zonal_stats_table(zone_raster, value_raster):
    mean_stats = []
    stdev_stats = []
    min_stats = []
    max_stats = []
    count_stats = []
    for i in np.unique(zone_raster):
        #something = np.where(zone_raster, i, np.nan)
        mean_stats.append(np.nanmean(np.concatenate([zone_raster, value_raster])))
        stdev_stats.append(np.nanstd(np.concatenate([zone_raster, value_raster])))
        min_stats.append(np.nanmin(np.concatenate([zone_raster, value_raster])))
        max_stats.append(np.nanmax(np.concatenate([zone_raster, value_raster])))
        count_stats.append(np.where(value_raster > 0, 1, 0).sum())
    zonal_stats = {'mean': mean_stats, 'stdev':stdev_stats, 'min':min_stats, 'max':max_stats, 'count':count_stats}
    df = pd.DataFrame(zonal_stats)
    output = df.to_csv(output_csv)
    return zonal_stats

In [7]:
zonal_stats_table(DEM_reclass, mean_coef)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)